In [1]:
from agent import QLearningAgent
from verify_data import FishGame, ParseError

In [2]:
with open('data/1-15_11:15.txt', 'r') as f:
    game = FishGame(f.readlines())
    game.rotate('PB')
    state = game.to_state()
with open('test.txt', 'w') as f:
    f.writelines([f'{row}\n' for row in state.tolist()])

In [3]:
import os

# TODO: implement shuffling of teammmates & opponents
memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-3_15:27.txt
12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
12-4_11:11.txt
1-15_11:15.txt


In [ ]:
agent = QLearningAgent()
agent.train_on_data(memories, [], 20)

In [ ]:
batch = agent.unpack_batch([memories[0]])
agent.q_network(agent.tensor(batch['state']), agent.action_masks(*batch['mask_dep'].values()))

In [ ]:
from datetime import datetime

os.makedirs('models', exist_ok=True)
model_path = f'models/fish_agent_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
agent.save_model(model_path)

In [5]:
agent = QLearningAgent(memories) 
agent.load_model('models/fish_agent_20250407_145324.pth')

/Users/Yourui/Documents/Fish/.env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


True

In [ ]:
agent.train_self_play(500, update_rate=5, epochs=5, path='models/fish_agent_20250406_213513.pth')

/Users/Yourui/Documents/Fish/agent.py:74: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.BoolTensor(x).to(self.device)


Game 0 finished, 6832 memories collected
Game 1 finished, 6832 memories collected
epoch 0, train loss 13.80706, test loss None, lr 0.0008
Game 2 finished, 6832 memories collected
epoch 0, train loss 10.8206, test loss None, lr 0.0008
Game 3 finished, 6832 memories collected
epoch 0, train loss 13.19224, test loss None, lr 0.0008
Game 4 finished, 6832 memories collected
epoch 0, train loss 13.5491, test loss None, lr 0.0008
Game 5 finished, 6832 memories collected
epoch 0, train loss 8.99714, test loss None, lr 0.0008
Game 6 finished, 6832 memories collected
epoch 0, train loss 11.43613, test loss None, lr 0.0008
Game 7 finished, 6832 memories collected
epoch 0, train loss 11.70345, test loss None, lr 0.0008
Game 8 finished, 6832 memories collected
epoch 0, train loss 10.44604, test loss None, lr 0.0008
Game 9 finished, 6832 memories collected
epoch 0, train loss 8.00655, test loss None, lr 0.0008
Game 10 finished, 6832 memories collected
epoch 0, train loss 5.59924, test loss None, lr 

KeyboardInterrupt: 